In [1]:
# Installazione pacchetti (da eseguire solo la prima volta)
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q qwen-omni-utils -U
!pip install -q bitsandbytes -U
!pip install -q accelerate
!pip install -q pandas
!pip install -q matplotlib # Aggiunto per la visualizzazione finale

# Import
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import re
import gc
import csv
import pandas as pd
import matplotlib.pyplot as plt

# --- CONFIGURAZIONE: Assicurati che questo sia il nome corretto del modello ---
model_name = "Qwen/Qwen3-4B"

# --- Preparazione della lista dei prompt dal CSV ---
# *** AGGIORNATO QUI: Percorso del file CSV di input con le descrizioni delle IMMAGINI ***
csv_file = "/kaggle/input/analysisimgdataset/analisi_immagini_qwen_2x2_con_metadati_max_tokens_512.csv"

prompt_list = []
results_for_csv = []

try:
    # Leggiamo il CSV con pandas.
    # Il CSV ha un header, quindi non usiamo 'header=None' e usiamo i nomi delle colonne.
    df = pd.read_csv(csv_file)
    
    # Riempi i valori NaN nella colonna 'Nome Immagine' con il valore precedente
    df['Nome Immagine'] = df['Nome Immagine'].fillna(method='ffill')

    # Raggruppa i dati per Nome Immagine
    for image_name, group in df.groupby('Nome Immagine'):
        descriptions = {}
        for _, row in group.iterrows():
            desc_type = row['Tipo Descrizione'].strip()
            desc_text = row['Descrizione'].strip()
            descriptions[desc_type] = desc_text

        # Costruisci l'elenco delle descrizioni nell'ordine desiderato (Frammento 1-4, poi Totale)
        # NUMERO DI FRAMMENTI: 4
        ordered_descriptions_content = []
        for i in range(1, 5): # Frammento 1 a 4 (range(1, 5) include 1, 2, 3, 4)
            content = descriptions.get(f'Frammento {i}', '')
            if content:
                ordered_descriptions_content.append({'label': f'Frammento {i}', 'content': content})

        # Aggiungi la descrizione totale per ultima
        overall_content = descriptions.get('Totale', '')
        if overall_content:
            ordered_descriptions_content.append({'label': 'Descrizione Totale', 'content': overall_content})

        # Assicurati di avere esattamente 5 descrizioni (4 frammenti + 1 totale)
        NUM_DESCRIPTIONS_EXPECTED = 5
        # Riempi con descrizioni vuote se mancano
        while len(ordered_descriptions_content) < NUM_DESCRIPTIONS_EXPECTED:
            missing_label = f"Mancante {len(ordered_descriptions_content) + 1}"
            if len(ordered_descriptions_content) == 4:
                missing_label = "Descrizione Totale Mancante"
            ordered_descriptions_content.append({'label': missing_label, 'content': ''})

        # Costruisci il prompt per il modello
        prompt = f"Nome Immagine: {image_name}\n\n"
        prompt += "Dati questi frammenti descrittivi di un'immagine, valuta su una scala da 0 a 255 quanto ciascuno dei seguenti stati emotivi è evocato dalla descrizione:\n"
        prompt += "agitazione/movimento (R), calma (G), tristezza (B).\n\n"
        prompt += f"Restituisci **ESATTAMENTE {NUM_DESCRIPTIONS_EXPECTED} triple RGB COMPLETE**, una dopo l'altra, su una SINGOLA riga. \n"
        prompt += "Non includere nessuna spiegazione, numerazione, elenchi puntati o altro testo aggiuntivo. Non usare la modalità di pensiero ('thinking mode').\n"
        prompt += f"Inizia la riga con 'RGB:' e poi le {NUM_DESCRIPTIONS_EXPECTED} triple. Ogni tripla deve contenere esattamente tre valori separati da virgole.\n\n"
        prompt += "Ecco l'ESEMPIO ESATTO del formato desiderato (solo 5 triple complete, su una riga):\n"
        prompt += "RGB: (R1, G1, B1) (R2, G2, B2) (R3, G3, B3) (R4, G4, B4) (R5, G5, B5)\n\n"
        prompt += "Ecco i frammenti:\n"
        for item in ordered_descriptions_content:
            if item['content']:
                prompt += f"- {item['label']}: {item['content']}\n"
        
        prompt_list.append({'image_name': image_name, 'prompt_text': prompt})

except FileNotFoundError:
    print(f"Errore: Il file CSV '{csv_file}' non è stato trovato.")
    print("Assicurati che il file si trovi nel percorso specificato.")
    exit()
except Exception as e:
    print(f"Si è verificato un errore durante la lettura del CSV: {e}")
    print("Assicurati che il CSV abbia le colonne 'Nome Immagine', 'Tipo Descrizione', 'Descrizione'.")
    exit()

# --- Fine preparazione della lista dei prompt ---


# Parametri modello
compute_dtype = torch.float16

# Configurazione quantizzazione 4-bit
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype
)

# Caricamento tokenizer e modello
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quant_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    torch_dtype=compute_dtype
)

# Loop sui prompt
for idx, entry in enumerate(prompt_list, start=1):
    image_name = entry['image_name']
    prompt_text = entry['prompt_text']

    print(f"\n=== Elaborazione immagine {idx}: {image_name} ===\n")

    chat = [{"role": "user", "content": prompt_text}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, enable_thinking=False)

    inputs = tokenizer(chat_text, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.1,
        num_return_sequences=1,
    )

    response_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    print("Risposta generata dal modello:\n")
    print(response_text)

    # --- MODIFICA REGEX E GESTIONE RGB ---
    rgb_value_pattern = re.compile(r"\((\d{1,3}),\s*(\d{1,3}),\s*(\d{1,3})\)")
    found_rgb_matches = rgb_value_pattern.findall(response_text)

    current_image_rgb_values = [image_name]

    # Popola fino a 5 valori RGB (4 frammenti + Totale).
    NUM_EXPECTED_RGB = 5
    for i in range(NUM_EXPECTED_RGB):
        if i < len(found_rgb_matches):
            try:
                r, g, b = int(found_rgb_matches[i][0]), int(found_rgb_matches[i][1]), int(found_rgb_matches[i][2])
                r, g, b = max(0, min(255, r)), max(0, min(255, g)), max(0, min(255, b))
                current_image_rgb_values.append(f"rgb({r}, {g}, {b})")
            except ValueError:
                print(f"Errore nel parsing RGB per il match {found_rgb_matches[i] if i < len(found_rgb_matches) else 'inatteso'}. Usando nero.")
                current_image_rgb_values.append("rgb(0, 0, 0)")
        else:
            current_image_rgb_values.append("rgb(0, 0, 0)")
            print(f"Avviso: Meno di {NUM_EXPECTED_RGB} triple RGB complete trovate per {image_name}. Aggiungendo nero per il frammento/descrizione {i+1}.")

    results_for_csv.append(current_image_rgb_values)

    # Pulisci la memoria GPU e CPU dopo ogni elaborazione
    del inputs, outputs
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# --- Salvataggio finale dei dati ---

print("\n--- Salvataggio dei colori RGB in CSV ---")
output_csv_file = "valutazione_rgb_immagini.csv"
csv_header = ['Nome Immagine', 'Frammento 1', 'Frammento 2', 'Frammento 3', 'Frammento 4', 'Descrizione Totale']

with open(output_csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    writer.writerows(results_for_csv)
print(f"Colori RGB salvati in '{output_csv_file}'")

print("\nProcessing complete. I valori RGB sono stati salvati nel file CSV.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 54.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00:00:0100:01


/tmp/ipykernel_35/921317915.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Nome Immagine'] = df['Nome Immagine'].fillna(method='ffill')


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...


=== Elaborazione immagine 98: wassily-kandinsky_improvisation-dreamy-1913.jpg ===

Risposta generata dal modello:

RGB: (200, 100, 50) (220, 150, 60) (180, 120, 40) (210, 130, 55) (190, 110, 45)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione immagine 99: william-henry-huddle_portrait-of-madam-candelaria-1891.jpg ===

Risposta generata dal modello:

RGB: (120, 160, 180) (140, 150, 130) (130, 140, 120) (110, 130, 100) (100, 120, 90)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione immagine 100: william-hogarth_before-the-seduction-and-after-1731.jpg ===

Risposta generata dal modello:

RGB: (170, 100, 50) (200, 150, 80) (180, 120, 60) (160, 110, 40) (190, 130, 70)

--- Salvataggio dei colori RGB in CSV ---
Colori RGB salvati in 'valutazione_rgb_immagini.csv'

Processing complete. I valori RGB sono stati salvati nel file CSV.


In [3]:
import csv
import re
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import ipywidgets as widgets
from IPython.display import display as ipy_display

def parse_rgb_to_normalized_tuple(rgb_string):
    """
    Converte una stringa come "rgb(r, g, b)" in una tupla di float (r, g, b)
    normalizzati nell'intervallo [0, 1] per matplotlib.
    Restituisce anche la tupla RGB originale (non normalizzata).
    """
    try:
        numeri = re.findall(r'\d+', rgb_string)
        if len(numeri) == 3:
            r_int, g_int, b_int = int(numeri[0]), int(numeri[1]), int(numeri[2])
            r_norm = r_int / 255.0
            g_norm = g_int / 255.0
            b_norm = b_int / 255.0
            return (r_norm, g_norm, b_norm), (r_int, g_int, b_int)
        else:
            return (0.0, 0.0, 0.0), (0, 0, 0)
    except (ValueError, IndexError):
        return (0.0, 0.0, 0.0), (0, 0, 0)

# --- FUNZIONE LEGENDA EMOZIONI ---
def print_color_emotion_legend():
    """
    Stampa una tabella/legenda delle emozioni mappate ai colori RGB,
    concentrandosi su quelle definite nel prompt del modello.
    """
    print("\n--- Legenda Emozioni e Componenti RGB ---")
    print("-" * 45)
    print(f"{'Componente RGB':<20} | {'Emozione Associata':<20}")
    print("-" * 45)
    print(f"{'R (Rosso)':<20} | {'Agitazione/Movimento':<20}")
    print(f"{'G (Verde)':<20} | {'Calma':<20}")
    print(f"{'B (Blu)':<20} | {'Tristezza':<20}")
    print("-" * 45)
    print("Nota: I colori finali sono combinazioni di questi valori primari.\n")


# --- FUNZIONE DI VISUALIZZAZIONE INTERATTIVA ---
def visualizza_colori_e_immagini_interattivo(csv_rgb_file, image_base_path, image_file_extension_list=[".jpg", ".jpeg", ".png", ".bmp"], max_immagini=None):
    """
    Legge un file CSV contenente i colori RGB, visualizza i colori e i loro valori RGB per ogni immagine
    e visualizza il relativo file immagine. L'utente deve cliccare un bottone per passare all'immagine successiva.
    """
    print(f"\n--- Inizio visualizzazione colori e immagini interattiva ---")
    print(f"Lettura colori da: '{csv_rgb_file}'")
    print(f"Cerca immagini in: '{image_base_path}' con estensioni: {', '.join(image_file_extension_list)}")

    data_to_visualize = []

    try:
        with open(csv_rgb_file, mode='r', newline='', encoding='utf-8') as file:
            lettore_csv = csv.reader(file)
            header = next(lettore_csv) # Salta l'intestazione
            for riga in lettore_csv:
                if not riga:
                    continue
                data_to_visualize.append(riga)

    except FileNotFoundError:
        print(f"ERRORE: Il file CSV '{csv_rgb_file}' non è stato trovato.")
        print("Assicurati che il percorso sia corretto e che il file esista.")
        return
    except Exception as e:
        print(f"Si è verificato un errore durante la lettura del CSV: {e}")
        return

    # Limita le immagini se max_immagini è specificato
    if max_immagini is not None:
        data_to_visualize = data_to_visualize[:max_immagini]

    if not data_to_visualize:
        print("Nessuna immagine trovata o da visualizzare. Controlla il CSV o i limiti impostati.")
        return

    current_image_index = 0
    output_area = widgets.Output() # Area di output per i contenuti dinamici
    button = widgets.Button(description="Prossima Immagine")

    def show_current_image():
        nonlocal current_image_index
        with output_area:
            clear_output(wait=True) # Pulisce l'output precedente
            if current_image_index < len(data_to_visualize):
                riga = data_to_visualize[current_image_index]
                nome_immagine = riga[0] # Nome Immagine dalla prima colonna
                colori_stringa = riga[1:6] # Le 5 colonne RGB

                colori_normalizzati_e_originali = [parse_rgb_to_normalized_tuple(s) for s in colori_stringa]
                colori_matplotlib = [item[0] for item in colori_normalizzati_e_originali]
                colori_originali_rgb = [item[1] for item in colori_normalizzati_e_originali]
                
                # --- Visualizzazione Matplotlib dei Colori ---
                fig, axes = plt.subplots(1, 5, figsize=(12.5, 3))
                fig.suptitle(f"Colori per l'Immagine: {nome_immagine}", fontsize=16)

                titoli = [f"Frammento {i+1}" for i in range(4)] + ["Descrizione Totale"]

                for i, (ax, colore_norm, colore_rgb, titolo) in enumerate(zip(axes, colori_matplotlib, colori_originali_rgb, titoli)): # Corretto titles a titoli
                    ax.imshow([[colore_norm]])
                    ax.set_title(titolo)
                    ax.set_xticks([])
                    ax.set_yticks([])

                    rgb_text = f"RGB: {colore_rgb}"
                    ax.text(0.5, -0.15, rgb_text, color='black', ha='center', va='top', transform=ax.transAxes, fontsize=10)

                plt.tight_layout(rect=[0, 0, 1, 0.9])
                plt.show() # Mostra la figura dei colori

                # --- Visualizzazione Immagine ---
                # Prova diverse estensioni se il file non è trovato con la prima
                image_full_path = None
                for ext in image_file_extension_list:
                    # Ho modificato questa riga per togliere l'estensione dal nome immagine prima di ricombinare
                    # Esempio: "my_image.jpg" -> "my_image" -> "my_image.ext"
                    base_name = os.path.splitext(nome_immagine)[0]
                    potential_path = os.path.join(image_base_path, f"{base_name}{ext}")
                    if os.path.exists(potential_path):
                        image_full_path = potential_path
                        break
                
                print(f"\n--- Immagine: {nome_immagine} ---")
                if image_full_path:
                    print(f"Tentativo di visualizzazione immagine: {image_full_path}")
                    try:
                        ipy_display(Image(filename=image_full_path))
                    except Exception as e:
                        print(f"ERRORE nella visualizzazione immagine per '{nome_immagine}': {e}")
                        print("Assicurati che il formato immagine sia supportato e che la libreria PIL sia installata.")
                else:
                    print(f"ATTENZIONE: File immagine NON TROVATO per '{nome_immagine}' al percorso base: {image_base_path}")
                    print(f"Verifica che il nome del file immagine corrisponda a '{nome_immagine}' nel CSV e che l'estensione sia corretta ({', '.join(image_file_extension_list)}).")
                
                current_image_index += 1
            else:
                print("Tutte le immagini sono state visualizzate!")
                button.disabled = True # Disabilita il bottone alla fine

    # Collega la funzione al click del bottone
    button.on_click(lambda b: show_current_image())

    # Visualizza il bottone e l'area di output
    ipy_display(button, output_area)

    # Mostra la prima immagine all'avvio
    show_current_image()

# --- Esempio di utilizzo dello script ---
if __name__ == "__main__":
    # --- CONFIGURA QUESTI PERCORSI E PARAMETRI ---
    
    # Nome del file CSV di output dalla PRIMA PARTE del codice
    output_rgb_csv_file = "valutazione_rgb_immagini.csv"

    # PERCORSO DELLA CARTELLA CHE CONTIENE LE TUE IMMAGINI ORIGINALI
    # AGGIORNATO CON IL TUO PATH SPECIFICATO
    image_source_directory = "/kaggle/input/imgdataset/images100"

    # Lista delle possibili estensioni dei tuoi file immagine.
    image_file_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"] 
    
    num_immagini_da_visualizzare = None # Limita a 3 per i test iniziali, cambia a None per tutte

    # --- Stampa la legenda prima di avviare l'interfaccia ---
    print_color_emotion_legend()

    visualizza_colori_e_immagini_interattivo(
        csv_rgb_file=output_rgb_csv_file,
        image_base_path=image_source_directory,
        image_file_extension_list=image_file_extensions,
        max_immagini=num_immagini_da_visualizzare
    )


--- Legenda Emozioni e Componenti RGB ---
---------------------------------------------
Componente RGB       | Emozione Associata  
---------------------------------------------
R (Rosso)            | Agitazione/Movimento
G (Verde)            | Calma               
B (Blu)              | Tristezza           
---------------------------------------------
Nota: I colori finali sono combinazioni di questi valori primari.


--- Inizio visualizzazione colori e immagini interattiva ---
Lettura colori da: 'valutazione_rgb_immagini.csv'
Cerca immagini in: '/kaggle/input/imgdataset/images100' con estensioni: .jpg, .jpeg, .png, .bmp, .tiff


Button(description='Prossima Immagine', style=ButtonStyle())

Output()